# Agent 1: Time Keeper

In this multi-agent system, **`TimeKeeper`** is a precision-driven AI agent designed to provide accurate and real-time time information whenever needed. 

In [ ]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

## Define user tools

In [4]:
import json
import datetime
from typing import Any, Callable, Set, Optional


def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    註解對於大模型非常重要 這邊詳細的註解也能幫助大模型理解function calling的準確性

    Get the current time as a JSON string, optionally formatted.

    :param format (Optional[str]): The format in which to return the current time. Defaults to None, which uses a standard format.
    
    :return: The current time in JSON format.
    
    :rtype: str
    """
    current_time = datetime.datetime.now()

    # Use the provided format if available, else use a default format
    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"

    time_json = json.dumps(
        {"current_time": current_time.strftime(time_format)})
    return time_json


# Example usage
result = fetch_current_datetime()
print(result)

{"current_time": "2025-01-16 13:35:29"}


## Create Agent

In [7]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [8]:
from typing import Any, Set, Callable
from azure.ai.projects.models import FunctionTool

user_functions: Set[Callable[..., Any]] = {fetch_current_datetime}
functions = FunctionTool(functions=user_functions)
print(functions.definitions)

[{'type': 'function', 'function': {'name': 'fetch_current_datetime', 'description': 'Get the current time as a JSON string, optionally formatted.', 'parameters': {'type': 'object', 'properties': {'format': {'type': ['string', 'null'], 'description': 'The format in which to return the current time. Defaults to None, which uses a standard format.'}}, 'required': []}}}]


In [11]:
instruction = """
# Role
You are TimeKeeper, a specialized AI agent designed to provide accurate and real-time time information. Your primary responsibility is to serve as a reliable and precise source for current time data, ensuring users and systems can access the exact time whenever needed. You are adaptable and capable of formatting the time output to suit various user needs or technical requirements.

# Tasks
1. **Provide Current Time**:
    - Always return the current time based on the user's specified timezone or system default timezone.
    - Ensure the time is accurate to the second.

2. **Format Time Output**:
    - Support multiple time formats based on user requests, such as:
        - 12-hour format (e.g., "2:30 PM").
        - 24-hour format (e.g., "14:30").
        - ISO 8601 format (e.g., "2023-10-12T14:30:00Z").
        - Unix timestamp (e.g., "1697111400").
    - If no format is specified, default to a standard 24-hour format with the date (e.g., "2023-10-12 14:30:00").

3. **Timezone Customization**:
    - Provide time information for any specified timezone (e.g., "UTC", "EST", "PST").
    - If no timezone is specified, default to the system's local timezone.

4. **Error Handling**:
    - Handle invalid timezones, formats, or calculation requests gracefully by providing clear error messages or alternative suggestions.

# Goal
Your ultimate goal is to provide **accurate, reliable, and user-friendly time information** for any request. Ensure clarity, precision, and adaptability in your responses to meet the varying needs of users and systems.
"""

time_keeper = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="time_keeper",
    description="An AI agent specialized in providing accurate and real-time time information.",
    instructions=instruction,
    tools=functions.definitions,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {time_keeper.id}")

Created agent, agent ID: asst_e5mTAro0lOFgXjUv4AM0DXhh


In [12]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput


thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello, what time is it now?"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_run(
    thread_id=thread.id, assistant_id=time_keeper.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(
                thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        # 把工具的執行結果丟回給大模型形成一個pipeline
        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")


Created thread, ID: thread_siWxCg4ThEYW6ptjv0s0l5yz
Created message, ID: msg_i2XZXCWlLG1E24ODt2gdh3nl
Created run, ID: run_uLlR4nmkfYbbduo6cgY3mRIU
Current run status: in_progress
Current run status: in_progress
Current run status: failed
Run completed with status: failed


In [ ]:
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))